In [1]:
import pandas as pd
import itertools
print(pd.DataFrame([['I1','I2','I5'],['I2','I4'],['I2','I3'],['I1','I2','I4'],['I1','I3'],['I2','I3'],['I1','I3'],['I1','I2','I3','I5'],['I1','I2','I3']]))

    0   1     2     3
0  I1  I2    I5  None
1  I2  I4  None  None
2  I2  I3  None  None
3  I1  I2    I4  None
4  I1  I3  None  None
5  I2  I3  None  None
6  I1  I3  None  None
7  I1  I2    I3    I5
8  I1  I2    I3  None


In [19]:
import pandas as pd
import itertools
'''
Load data
'''
def loadSimpDat():
    simpDat = [['I1','I2','I5'],['I2','I4'],['I2','I3'],['I1','I2','I4'],['I1','I3'],['I2','I3'],['I1','I3'],['I1','I2','I3','I5'],['I1','I2','I3']]
    '''simpDat=[]
    with open('Example_data.txt', 'r') as f:
        for line in f.readlines():
            L=line.split(',')
            L[-1]=L[-1].replace('\n','')
            simpDat.append(L)'''
    return simpDat

#tree node
class TreeNode:
    def __init__(self, item, count, parentNode):
        self.item=item
        self.count=count
        self.nodelink=None
        self.parent=parentNode
        self.children={}
    def inc_count(self,num):
        self.count+=num

'''
Generate Headertable and remove item does not meet minimum support
|item|counts|Nodelink
'''
def GenC1(trans,min_support,total):
    C1={}
    for i in trans:
        for ii in set(i):
            if ii not in C1:
                try:
                    C1[ii]=trans[i]
                except:
                    C1[ii]=1
            else:
                try:
                    C1[ii]+=trans[i]
                except:
                    C1[ii]+=1
    C1={key:value for key, value in C1.items() if value>=min_support*total}#include min_support
    print('C1',C1)
    return {i:[k,None] for i,k in sorted(C1.items(),key=lambda x:x[1],reverse=True)}

def link_same_items(curr,targetNode):
    while curr.nodelink is not None:
        curr=curr.nodelink
    curr.nodelink=targetNode


'''
Build Tree
'''
def link_same_items(curr,targetNode):
    while curr.nodelink is not None:
        curr=curr.nodelink
    curr.nodelink=targetNode

def updateTree_Table(SortedItems,FPTree,headertable,count):
    if SortedItems[0] in FPTree.children:
        FPTree.children[SortedItems[0]].inc_count(count)
    else:
        FPTree.children[SortedItems[0]]=TreeNode(SortedItems[0],count,FPTree)
        if headertable[SortedItems[0]][1]==None:
            headertable[SortedItems[0]][1]=FPTree.children[SortedItems[0]]
        else:
            link_same_items(headertable[SortedItems[0]][1], FPTree.children[SortedItems[0]])
    if len(SortedItems)>1:
        updateTree_Table(SortedItems[1:],FPTree.children[SortedItems[0]],headertable,count)

def Build_tree(trans,headertable,count=1):
    recTree=TreeNode('Null',1,None)
    for iii,itemset in enumerate(trans):
        #order the itemset according to the header table
        itemset=[i for i in itemset if i in headertable]
        if len(itemset)>0:
            SortedItems=sorted(list(itemset),key=lambda x:headertable[x][0],reverse=True)
            try:
                updateTree_Table(SortedItems,recTree,headertable,count[iii])
            except:
                updateTree_Table(SortedItems,recTree,headertable,count)
    return recTree,headertable

'''
Mine Tree
'''
def ascendTree(leafNode,PrefixPath):
    if leafNode.parent is not None:
        PrefixPath.append(leafNode.item)
        ascendTree(leafNode.parent, PrefixPath)

def findPrefixPath(baseitem,TreeNode):
    CPB={}
    while TreeNode is not None:
        PrefixPath=[]
        ascendTree(TreeNode,PrefixPath)
        if len(PrefixPath)>1:
            CPB[frozenset(PrefixPath[1:])]=TreeNode.count
        TreeNode=TreeNode.nodelink
    return CPB

def mineTree(inTree, headertable, min_support, preFix, freqItemList,total):
    reverse_headertable = [v for v,_ in sorted(headertable.items(), key=lambda x: x[1][0])]
    for baseitem in reverse_headertable:
        newFreqSet = preFix.copy()
        newFreqSet.add(baseitem)
        freqItemList.append(newFreqSet)
        #print('newFreqSet:',newFreqSet)
        #print('Freqitemls:',freqItemList)

        # Conditional Pattern Base
        CPB = findPrefixPath(baseitem, headertable[baseitem][1])
        #if len(CPB)!=0:
            #print(pd.DataFrame({'Item':[str(baseitem) for i in range(len(CPB))],'Prefix':[(i) for i in CPB],'count':[(i) for i in CPB.values()]}))
        CPBC1=GenC1(CPB,min_support,total)
        #if len(CPBC1)!=0:
            #print(CPBC1)
            #print('-'*30)
        CondFPTree, Condheadertable = Build_tree(list(CPB.keys()),CPBC1,list(CPB.values()))
        if Condheadertable is not None:
            #myCondTree.disp(1)
            mineTree(CondFPTree, Condheadertable, min_support, newFreqSet, freqItemList,total)


'''
Generate Support Number
'''
def calSuppData(FPtree,headerTable, freqItemList):
    suppData = {}
    for Item in freqItemList:
        base={}
        #print('*'*10)
        #print(Item)
        #print('*'*10)
        # Search from smallest to largest
        Item = sorted(Item, key=lambda x:headerTable[x][0])
        for ii in range(len(Item)):
            prebase = findPrefixPath(Item[ii], headerTable[Item[ii]][1])
            #print('Item:',Item[ii],'Base Before:', prebase)

            prebase={frozenset(set(each_base).union({Item[ii]})):value for each_base,value in prebase.items()}
            if Item[ii] in FPtree.children.keys():
                prebase[Item[ii]]=FPtree.children[Item[ii]].count+prebase.get(Item[ii],0)
            #print('Base After:', prebase)
            #print('Item:',Item[ii],'preBase:', prebase)
            base.update(prebase)
            #print('Base:',base)

        #print('-----Start to count:')
        support = 0
        for B in base:
            #print('Item:',frozenset(Item))
            #print('B:',B,base[B])
            if not isinstance(B,str):
                set_B=set(B)
            else:
                set_B={B}
            if frozenset(Item).issubset(set_B):
                support += base[B]
            #print('Support:',support)
        #print('-'*40)

        suppData[frozenset(Item)] = support
    return suppData


'''
Generate Confidence
'''
def calconf(FPtree,headerTable,freqItemList,Support,min_conf):
    allrules=[]
    for itemset in freqItemList:
        S=Support[frozenset(itemset)]
        check_leng=len(itemset)-1
        if check_leng==0:
            pass
            #allrules.append((itemset,itemset,'nan',headerTable[list(itemset)[0]][0],S,'nan'))
            #print(itemset,':',headerTable[list(itemset)[0]][0])
            #print(itemset,'--->','Nothing','Confidence:',headerTable[list(itemset)[0]][0],'Support:',S)
        for i in range(check_leng):
            n=i+1
            Test=[set(i) for i in itertools.combinations(itemset,n)]
            Rest=[itemset-i for i in Test]
            #print('Test:',Test)
            #print('Rest:',Rest)
            try:
                Confidence=calSuppData(FPtree,headerTable,Test)
                supportB=calSuppData(FPtree,headerTable,Rest)
                #print('Condidence:',Confidence)
                Confidence_value=[Confidence[frozenset(i)] for i in Test]
                supportB_value=[supportB[frozenset(i)] for i in Rest]
                #print('Confidence_value:',Confidence_value)
                for rule in range(len(Test)):
                    if Confidence_value[rule]*(min_conf)<=S:
                        allrules.append((itemset,Test[rule],Rest[rule],Confidence_value[rule],S,supportB_value[rule]))
                        #print(Test[rule],'--->',Rest[rule],'Confidence:',Confidence_value[rule],'Support:',S)
            except:
                raise
            #print('-'*50)
    return allrules
    
def FP(simpDat,min_support,total):
    C1=GenC1(simpDat,min_support,total)
    myTree,myHeader=Build_tree(simpDat,C1)
    freqItemList,preFix=[],set([])
    mineTree(myTree, myHeader, min_support, preFix, freqItemList,total)
    Support=calSuppData(myTree,myHeader,freqItemList)#{Frozenset:count}
    return freqItemList,Support, myTree,myHeader

def printsupport(S):
    S_table=pd.DataFrame.from_dict(S,orient='index').reset_index()
    S_table.columns=['itemset','support']
    S_table
    return S_table

def printrules(rules,total):
    C=['Frequent Itemset(AUB)','From(A)','To(B)','Confident Number(num of A)','Support Number(num of AUB)','Support Number(num of B)']
    Ruletable=pd.DataFrame(rules)
    Ruletable.columns=C
    Ruletable['Confidence']=Ruletable['Support Number(num of AUB)']/Ruletable['Confident Number(num of A)']
    Ruletable['Support']=Ruletable['Support Number(num of AUB)']/total
    Ruletable['Lift']=(Ruletable['Support Number(num of AUB)']/total)/(Ruletable['Support Number(num of B)']/total)/(Ruletable['Confident Number(num of A)']/total)
    return Ruletable

if __name__=='__main__':
    simpDat=loadSimpDat()
    min_support=1.9/9 #*len(simpDat)
    freqItemList,Support,myTree,myHeader=FP(simpDat,min_support,len(simpDat))

    #Support[frozenset([])] = 1.0
    #freqItems = [frozenset(x) for x in freqItemList]
    print(freqItemList)
    min_conf=0.5
    Conf=calconf(myTree,myHeader,freqItemList,Support,min_conf)

NameError: name 'total' is not defined

In [16]:
printsupport(Support)

,itemset,support
0,(I5),2
1,"(I3, I5)",1
2,"(I3, I5, I1)",1
3,"(I3, I5, I2)",1
4,"(I3, I5, I1, I2)",1
5,"(I5, I1)",2
6,"(I5, I2)",2
7,"(I5, I1, I2)",2
8,(I4),2
9,"(I1, I4)",1


In [17]:
TY=printrules(Conf,len(simpDat))
TY.sort_values('Frequent Itemset(AUB)',ascending=False).reset_index(drop=True)

,Frequent Itemset(AUB),From(A),To(B),Confident Number(num of A),Support Number(num of AUB),Support Number(num of B),Confidence,Support,Lift
0,"{I3, I5, I1, I2}","{I5, I2}","{I3, I1}",2,1,4,0.500000,0.111111,1.125000
1,"{I3, I5, I1, I2}","{I3, I5, I2}",{I1},1,1,6,1.000000,0.111111,1.500000
2,"{I1, I4, I2}","{I1, I4}",{I2},1,1,7,1.000000,0.111111,1.285714
3,"{I1, I4, I2}","{I4, I2}",{I1},2,1,6,0.500000,0.111111,0.750000
4,"{I4, I2}",{I4},{I2},2,2,7,1.000000,0.222222,1.285714
5,"{I3, I5, I1, I2}","{I5, I1, I2}",{I3},2,1,6,0.500000,0.111111,0.750000
6,"{I3, I5, I1, I2}",{I5},"{I3, I1, I2}",2,1,2,0.500000,0.111111,2.250000
7,"{I3, I5, I1, I2}","{I3, I5}","{I1, I2}",1,1,4,1.000000,0.111111,2.250000
8,"{I3, I5, I1, I2}","{I5, I1}","{I3, I2}",2,1,4,0.500000,0.111111,1.125000
9,"{I3, I5, I1, I2}","{I3, I1, I2}",{I5},2,1,2,0.500000,0.111111,2.250000


In [13]:
total=len(simpDat)
C=['Frequent Itemset(AUB)','From(A)','To(B)','Confident Number(num of A)','Support Number(num of AUB)','Support Number(num of B)']
Ruletable=pd.DataFrame(Conf)
Ruletable.columns=C
Ruletable['Confidence']=Ruletable['Support Number(num of AUB)']/Ruletable['Confident Number(num of A)']
Ruletable['Support']=Ruletable['Support Number(num of AUB)']/total
#Ruletable['Lift']=[Ruletable['Support Number(num of AUB)']/Ruletable['Support Number(num of B)'] for i in range(len(Ruletable['Support'])) if Ruletable['Support Number(num of B)']!='nan']
#print(len(Ruletable['Support']))
#Ruletable['Lift']=[i for i in range(len(Ruletable['Support'])) if Ruletable.loc[i,'Support Number(num of B)']!='nan']
Ruletable.head()

,Frequent Itemset(AUB),From(A),To(B),Confident Number(num of A),Support Number(num of AUB),Support Number(num of B),Confidence,Support
0,"{I2, I5}",{I2},{I5},7,2,2,0.285714,0.222222
1,"{I2, I5}",{I5},{I2},2,2,7,1.000000,0.222222
2,"{I1, I5}",{I1},{I5},6,2,2,0.333333,0.222222
3,"{I1, I5}",{I5},{I1},2,2,6,1.000000,0.222222
4,"{I2, I4}",{I2},{I4},7,2,2,0.285714,0.222222


In [10]:
C=['Frequent itemset','From','To','Confident number','Support Number']
Ruletable=pd.DataFrame(Conf)
Ruletable.columns=C
Ruletable['Confidence']=Ruletable['Support Number']/Ruletable['Confident number']
Ruletable.sort_values('Confident number',ascending=False)

,Frequent itemset,From,To,Confident number,Support Number,Confidence
28,{I2},{I2},nan,7,7,1.000000
1,"{I2, I5}",{I2},{I5},7,2,0.285714
22,"{I2, I1, I3}",{I2},"{I1, I3}",7,2,0.285714
5,"{I2, I1, I5}",{I2},"{I1, I5}",7,2,0.285714
18,"{I2, I3}",{I2},{I3},7,4,0.571429
12,"{I2, I4}",{I2},{I4},7,2,0.285714
15,"{I2, I1}",{I2},{I1},7,4,0.571429
24,"{I2, I1, I3}",{I3},"{I2, I1}",6,2,0.333333
23,"{I2, I1, I3}",{I1},"{I2, I3}",6,2,0.333333
21,"{I1, I3}",{I3},{I1},6,4,0.666667


In [51]:
print(Support)
print(pd.DataFrame(myHeader.keys()))
#print(pd.DataFrame(C1.values(),C1.keys()))
'''a={'I1'}
print(myHeader[list(a)[0]])'''

{frozenset({'I5'}): 2, frozenset({'I1', 'I5'}): 2, frozenset({'I2', 'I5'}): 2, frozenset({'I1', 'I2', 'I5'}): 2, frozenset({'I4'}): 2, frozenset({'I2', 'I4'}): 2, frozenset({'I1'}): 6, frozenset({'I1', 'I2'}): 4, frozenset({'I3'}): 6, frozenset({'I3', 'I2'}): 4, frozenset({'I1', 'I3', 'I2'}): 2, frozenset({'I1', 'I3'}): 4, frozenset({'I2'}): 7, frozenset(): 1.0}
    0
0  I2
1  I1
2  I3
3  I5
4  I4


"a={'I1'}\nprint(myHeader[list(a)[0]])"

In [15]:
print(pd.DataFrame(freqItemList))

     0     1     2
0   I5  None  None
1   I2    I5  None
2   I5    I1  None
3   I2    I5    I1
4   I4  None  None
5   I2    I4  None
6   I1  None  None
7   I2    I1  None
8   I3  None  None
9   I3    I2  None
10  I3    I1  None
11  I3    I2    I1
12  I2  None  None


In [16]:
A=pd.DataFrame({'FredP':Support,'Support:':Support.values}).reset_index()
#A['leng']=[len(i) for i in A['index']]
#A.sort_values('leng').reset_index(drop=True)
A.sort_values('FredP').reset_index()

,level_0,index,FredP,Support:
0,0,(I5),2,<built-in method values of dict object at 0x14...
1,1,"(I2, I5)",2,<built-in method values of dict object at 0x14...
2,2,"(I5, I1)",2,<built-in method values of dict object at 0x14...
3,3,"(I2, I5, I1)",2,<built-in method values of dict object at 0x14...
4,4,(I4),2,<built-in method values of dict object at 0x14...
5,5,"(I2, I4)",2,<built-in method values of dict object at 0x14...
6,12,"(I3, I2, I1)",2,<built-in method values of dict object at 0x14...
7,7,"(I2, I1)",4,<built-in method values of dict object at 0x14...
8,10,"(I3, I2)",4,<built-in method values of dict object at 0x14...
9,11,"(I3, I1)",4,<built-in method values of dict object at 0x14...


In [17]:
transactions=loadSimpDat()
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
#df.head()
from mlxtend.frequent_patterns import fpgrowth
FP=fpgrowth(df, min_support=1.9/9,use_colnames=True)
FP['supportnum']=FP['support']*len(transactions)
FP['leng']=[len(i) for i in FP['itemsets']]
FP.sort_values('support',ascending=True).reset_index(drop=True)

,support,itemsets,supportnum,leng
0,0.222222,(I5),2.0,1
1,0.222222,(I4),2.0,1
2,0.222222,"(I3, I2, I1)",2.0,3
3,0.222222,"(I5, I1)",2.0,2
4,0.222222,"(I2, I5)",2.0,2
5,0.222222,"(I2, I5, I1)",2.0,3
6,0.222222,"(I2, I4)",2.0,2
7,0.444444,"(I2, I1)",4.0,2
8,0.444444,"(I3, I1)",4.0,2
9,0.444444,"(I3, I2)",4.0,2


In [55]:
idx=[]
for m,ii in enumerate([set(i) for i in FP.itemsets]):
    for j in A.FredP:
        if j==(ii):
            idx.append(m)
[k for i,k in enumerate([set(i) for i in FP.itemsets]) if i not in set(idx)]

[{'I2'},
 {'I1'},
 {'I5'},
 {'I4'},
 {'I3'},
 {'I1', 'I2'},
 {'I1', 'I3'},
 {'I1', 'I2', 'I3'},
 {'I1', 'I5'},
 {'I2', 'I5'},
 {'I1', 'I2', 'I5'},
 {'I2', 'I4'},
 {'I2', 'I3'}]

In [51]:
#reservior for original code

def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1 == L2: 
                retList.append(Lk[i] | Lk[j])
    return retList

def calcConf(freqSet, H, supportData, br1, minConf=0.7):
    prunedH = []
    for conseq in H:
        print('ConfreqSet:',freqSet,supportData[freqSet])
        print('Conf conseq:',conseq,supportData[conseq])
        print('Conf [freqSet - conseq]:',[freqSet - conseq],supportData[freqSet - conseq])

        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print ("{0} --> {1} conf:{2}".format(freqSet - conseq, conseq, conf))
            br1.append((freqSet - conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, br1, minConf=0.7):
    print('Rule freqSet',freqSet)
    m = len(H[0])
    if len(freqSet) > m+1:
        Hmp1 = aprioriGen(H, m+1)
        print('Hmp1:',Hmp1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, br1, minConf)
        if len(Hmp1)>1:
            rulesFromConseq(freqSet, Hmp1, supportData, br1, minConf)

def generateRules(freqItemList, supportData, minConf=0.7):
    bigRuleList = []
    for freqSet in freqItemList:
        H1 = [frozenset([item]) for item in freqSet]
        print('H1:',H1)
        if len(freqSet)>1:
            rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
        else:
            calcConf(freqSet, H1, supportData, bigRuleList, minConf)
        print('-'*30)
    return bigRuleList
